In [1]:
resource = "../../data/generated/"
results = "../../results/"

In [2]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
from linearmodels.panel import PanelOLS, compare
import statsmodels.formula.api as smf
# from statsmodels.iolib.summary2 import summary_col
# from pystout import pystout
from stargazer.stargazer import Stargazer
from tabprintin.beautify import *

In [3]:
smp = pd.read_csv(resource + "acc_us_sample.csv")
smp.replace([np.inf, -np.inf], np.nan, inplace=True)
smp_da = smp[['gvkey', 'fyear', 'ff12_ind', 'mj_da', 'dd_da', 'ln_ta', 
              'ln_mktcap', 'mtb', 'ebit_avgta', 'sales_growth']]
smp_da = smp_da.dropna(subset=['fyear', 'mj_da', 'dd_da', 'ln_ta', 'ln_mktcap', 'mtb', 'ebit_avgta', 'sales_growth'])

list_to_win = ['mj_da', 'dd_da', 'ln_ta', 'ln_mktcap', 'mtb', 'ebit_avgta', 'sales_growth']
for i in list_to_win:
    smp_da[i] = smp_da.groupby('fyear')[i].transform(lambda x: winsorize(x,limits=[.01,.01]))

In [11]:
smp_da.to_csv('smp_da.csv')

# Summary statistics

In [4]:
var_name = (['Modified Jones DA', 
             'Dechow and Dichev DA', 
             'Ln(Total assets)', 
             'Ln(Market capitalization)',
             'Market to book', 
             'Return on assets', 
             'Sales growth'])

stat_name = (['N', 
             'Mean', 
             'Std. dev.', 
             'Min.', 
             '25\%', 
             'Median',
             '75\%',
             'Max.'])

summary_stats = smp_da[['mj_da', 'dd_da', 'ln_ta', 'ln_mktcap', 'mtb', 'ebit_avgta', 'sales_growth']].describe().T

summary_stats.index = var_name
summary_stats.columns = stat_name

summary_stats.N = summary_stats.N.astype(int)

print(summary_stats.style.format(decimal='.', thousands=',', precision=3).to_latex())
summary_stats

\begin{tabular}{lrrrrrrrr}
 & N & Mean & Std. dev. & Min. & 25\% & Median & 75\% & Max. \\
Modified Jones DA & 57,163 & 0.006 & 0.237 & -1.927 & -0.047 & 0.017 & 0.088 & 1.027 \\
Dechow and Dichev DA & 57,163 & 0.001 & 0.096 & -0.490 & -0.032 & 0.002 & 0.038 & 0.385 \\
Ln(Total assets) & 57,163 & 4.568 & 2.403 & -2.207 & 2.871 & 4.466 & 6.219 & 11.770 \\
Ln(Market capitalization) & 57,163 & 4.558 & 2.421 & -1.552 & 2.794 & 4.439 & 6.284 & 11.924 \\
Market to book & 57,163 & 2.786 & 8.532 & -125.299 & 0.929 & 1.801 & 3.485 & 123.185 \\
Return on assets & 57,163 & -0.125 & 0.643 & -12.152 & -0.104 & 0.042 & 0.088 & 0.897 \\
Sales growth & 57,163 & 1.013 & 0.471 & -4.642 & 0.988 & 1.013 & 1.055 & 5.902 \\
\end{tabular}



,N,Mean,Std. dev.,Min.,25\%,Median,75\%,Max.
Modified Jones DA,57163,0.006109,0.237157,-1.926686,-0.047093,0.017298,0.087554,1.026649
Dechow and Dichev DA,57163,0.001120,0.095979,-0.489783,-0.032441,0.001534,0.038344,0.385274
Ln(Total assets),57163,4.568192,2.402711,-2.207275,2.871331,4.466322,6.219185,11.770022
Ln(Market capitalization),57163,4.557544,2.421399,-1.552481,2.793788,4.439238,6.283627,11.924330
Market to book,57163,2.786015,8.532028,-125.299106,0.928928,1.801402,3.485184,123.184914
Return on assets,57163,-0.125160,0.642789,-12.152318,-0.104129,0.041673,0.088155,0.896939
Sales growth,57163,1.012775,0.471146,-4.642257,0.987810,1.013323,1.055299,5.902028


# Regression

In [5]:
mod1 = PanelOLS.from_formula('mj_da ~ EntityEffects + TimeEffects + ln_ta + mtb + ebit_avgta + sales_growth',
                            data=smp_da.set_index(['gvkey', 'fyear'])
                           )
reg1 = mod1.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

mod2 = PanelOLS.from_formula('dd_da ~ EntityEffects + TimeEffects + ln_ta + mtb + ebit_avgta + sales_growth',
                            data=smp_da.set_index(['gvkey', 'fyear'])
                           )
reg2 = mod2.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

In [6]:
print(reg1.summary.tables[1],reg2.summary.tables[1])

                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
ebit_avgta       0.2322     0.0258     8.9914     0.0000      0.1816      0.2828
ln_ta           -0.0410     0.0052    -7.9027     0.0000     -0.0512     -0.0308
mtb           2.526e-05     0.0002     0.1172     0.9067     -0.0004      0.0004
sales_growth     0.0098     0.0043     2.2714     0.0231      0.0013      0.0182
================================================================================                               Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
ebit_avgta       0.0492     0.0079     6.2288     0.0000      0.0337      0.0647
ln_ta            0.0035     

In [7]:
print(compare({'model_A_name': reg1, 'model_B_name': reg2 }, stars = True))

                  Model Comparison                  
                          model_A_name  model_B_name
----------------------------------------------------
Dep. Variable                    mj_da         dd_da
Estimator                     PanelOLS      PanelOLS
No. Observations                 57163         57163
Cov. Est.                    Clustered     Clustered
R-squared                       0.1890        0.0590
R-Squared (Within)              0.1752        0.0587
R-Squared (Between)            -0.6533       -0.0607
R-Squared (Overall)            -0.6454       -0.0257
F-statistic                     2811.9        756.65
P-value (F-stat)                0.0000        0.0000
=====================     ============   ===========
ebit_avgta                   0.2322***     0.0492***
                              (8.9914)      (6.2288)
ln_ta                       -0.0410***     0.0035***
                             (-7.9027)      (2.9030)
mtb                          2.526e-05     0.0

In [8]:
prova3 = smf.ols('dd_da ~ ln_ta + mtb + ebit_avgta + sales_growth',data=smp_da).fit()

In [9]:
prova = pystout(models=[reg1,reg2,prova3],
        file='test_table.tex',
        addnotes=['Here is a little note','And another one'],
        digits=3,
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}','fvalue':'F-stat'}
        )
print(prova)


\hline
Obs & 57163 & 57163 & 57163 \\
Adj. R\sym{2} &  &  & 0.054 \\
F-stat & 2811.931 & 756.648 & 812.885 \\
\hline\hline
\multicolumn{4}{l}{\footnotesize Here is a little note}\vspace{-.25em} \\
\multicolumn{4}{l}{\footnotesize And another one}
\end{tabular}
}


C:\Users\Huang\OneDrive - Università Commerciale Luigi Bocconi (1)\python\reprotin\code\do_analysis\tabprintin\beautify.py:376: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(pd.DataFrame([r],index=[value]))
C:\Users\Huang\OneDrive - Università Commerciale Luigi Bocconi (1)\python\reprotin\code\do_analysis\tabprintin\beautify.py:376: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(pd.DataFrame([r],index=[value]))
C:\Users\Huang\OneDrive - Università Commerciale Luigi Bocconi (1)\python\reprotin\code\do_analysis\tabprintin\beautify.py:376: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(pd.DataFrame([r],index=[value]))


In [10]:
prova

'\n\\hline\nObs & 57163 & 57163 & 57163 \\\\\nAdj. R\\sym{2} &  &  & 0.054 \\\\\nF-stat & 2811.931 & 756.648 & 812.885 \\\\\n\\hline\\hline\n\\multicolumn{4}{l}{\\footnotesize Here is a little note}\\vspace{-.25em} \\\\\n\\multicolumn{4}{l}{\\footnotesize And another one}\n\\end{tabular}\n}'